# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# options.add_argument('headless') # 창 숨기기


In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.185 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.81 Safari/537.36


In [3]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


C:\Users\user\AppData\Local\Temp\ipykernel_15164\1335855278.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


In [4]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_8월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(12508, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/yoosoojoung/222076684802,이마트 트레이더스 월계점 가성비갑 대형식품,2020.08.31.
1,https://blog.naver.com/yuneyun/222076685632,<백일독서 그물독서#1>마이퍼스트월드: 바퀴,2020.08.31.
2,https://blog.naver.com/jun95990/222076683798,[포스코] 2020 하반기 포스코 채용,2020.08.31.
3,https://blog.naver.com/iloveyou915/222076683007,광주광역시 서구 상무시민공원 시민들의 휴식공간으로 딱이야!,2020.08.31.
4,https://blog.naver.com/mkw728/222076680205,(탄핵 무상)자비의 샘물-55,2020.08.31.
...,...,...,...
12503,https://blog.naver.com/imjwimjw/222047769668,신창세론: 20 장 시대의 요청,2020.08.01.
12504,https://blog.naver.com/whynews1/222047767476,[인터뷰] 생각하고 ‘신속’ 행동하는 정치인 한은경 오산시의원,2020.08.01.
12505,https://blog.naver.com/alpepel/222047765522,광주광역시동물병원 고정을 시켜놓고 올림,2020.08.01.
12506,https://blog.naver.com/sheenbee/222047763422,남파랑길 부산 5코스(신평동 교차로~강서구 송정공원),2020.08.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


12508


In [6]:
df = pd.DataFrame(columns = {'url','본문'})
df.to_csv("./나무문화/2020/나무문화_본문_2020_8월.csv", index=False, encoding='UTF-8')
df.to_excel("./나무문화/2020/나무문화_본문_2020_8월.xlsx", index=False, encoding='UTF-8')

In [9]:
len(df)

1998

In [8]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_8월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_8월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    if blog_links.index(i) != df['본문'].tolist().index(a):
        break


  0%|          | 0/12508 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_15164\2772954805.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 

c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://youwin0427.tistory.com/1770
서범석%20나이%20뮤지컬배우%20결혼%20부인%20아내%20와이프%20자녀%20가족%20고향%20군대
탤런트%20영화%20뮤지컬%20배우%20서범석%20프로필%20서범석%20나이%20키%20학력%20고향%20종교%20소속사%20인스타%20몸무게%20이름%20서범석%20국적%20대한민국%20출생%201970년%208월%2017일,%20서울특별시%20나이%2050세%20종교%20개신교%20직업%20배우%20&%20뮤지컬%20배우..
youwin0427.tistory.com
탤런트%20영화%20뮤지컬%20배우%20서범석%20프로필
서범석%20나이%20키%20학력%20고향%20종교%20소속사%20인스타%20몸무게
이름
서범석
국적
대한민국
출생
1970년%208월%2017일,%20서울특별시
나이
50세
종교
개신교
직업
배우%20&%20뮤지컬%20배우
신장
178cm
학력
중앙대학교%20산업정보학과
서범석%20뮤지컬배우
출생
1970년%208월%2017일
신체
178cm,%2068kg
학력
중앙대학교%20산업정보학%20학사
수상
2009년%20제3회%20대구국제뮤지컬페스티벌%20남우주연상%20(라디오스타)
2008년%20제14회%20한국뮤지컬대상%20남우조연상%20(노트르담%20드%20파리)
2008년%20제2회%20대구뮤지컬어워즈%20최고스타상%20(노트르담%20드%20파리)
사이트
공식홈페이지,%20미니홈피
출생:%201970년%208월%2017일%20(50세),%20대한민국
학력:%20중앙대학교%20서울캠퍼스
서범석(1970년%208월%2017일1%20~)은%20대한민국의%20뮤지컬%20배우이다.서범석%20종교는%20개신교이다.%20《명성황후》,《노트르담%20드%20파리》,%20《라디오스타》%20등%20다수의%20뮤지컬에%

171 171
172 172
173 173
174 174
175 175
176 176
177 177
178 178
179 179
180 180
181 181
182 182
183 183
184 184
185 185
186 186
187 187


...

c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://blog.naver.com/bigeyes202/222068727004
코로나%202차%20유행에%20따른%20아직%20오르지%20않은%20수혜주%20를%20알아보자%20!
https://www.nocutnews.co.kr/news/5399184코로나%202차%20팬더믹이%20시작이%20되었다.사랑제일교회발%20확산이%20전국...
blog.naver.com
짭짤한%20한%20주%20였네요.
저번주에%20정리해둔%20자료가
거리두기%202.5단계%20상승에%20따라%20시세를%20주기%20시작했습니다.
동방이%20주력종목이었는데%20평단%201800원에서%2025일%20한번%20하락에%20털린후%202050원에%20다시%20잡아서%20생각보다%20아쉬웠습니다.%2030%25이상%20먹을%20수%20있었는데%20ㅠ_ㅠ
그리고%20' with link or location/anchor > 2079 characters since it exceeds Excel's limit for URLS
  warn("Ignoring URL '%s' with link or location/anchor > %d "


861 861
862 862
863 863
864 864
865 865
866 866
867 867
868 868
869 869
870 870
871 871
872 872
873 873
874 874
875 875
876 876
877 877
878 878
879 879
880 880
881 881
882 882
883 883
884 884
885 885
886 886
887 887
888 888
889 889
890 890
891 891
892 892
893 893
894 894
895 895
896 896
897 897
898 898
899 899
900 900
901 901
902 902
903 903
904 904
905 905
906 906
907 907
908 908
909 909
910 910
911 911
912 912
913 913
914 914
915 915
916 916
917 917
918 918
919 919
920 920
921 921
922 922
923 923
924 924
925 925
926 926
927 927
928 928
929 929
930 930
931 931
932 932
933 933
934 934
935 935
936 936
937 937
938 938
939 939
940 940
941 941
942 942
943 943
944 944
945 945
946 946
947 947
948 948
949 949
950 950
951 951
952 952
953 953
954 954
955 955
956 956
957 957
958 958
959 959
960 960
961 961
962 962
963 963
964 964
965 965
966 966
967 967
968 968
969 969
970 970
971 971
972 972
973 973
974 974
975 975
976 976
977 977
978 978
979 979
980 980
981 981
982 982
983 983
984 984
985 985


c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://komayotq.blog.me/222074145057
20180116%20서리%20in%20New%20Zealand%20(1)
코로나로%20집에만%20콕%20박혀있는%20요즘,%20자꾸만%20어디론가%20떠나고%20싶은%20마음이%20들기%20마련입니다.사계절%20중%20여...
komayotq.blog.me
-앞서%20작성했던%20약%202주간의%20뉴질랜드%20일정에%20이어서.
퀸즈타운에서%20보냈던%20하루하루는%20여름이라는%20계절이%20더욱%20특별하게%20느껴지게끔%20해주었고,%20제게%20하나의%20로망을%20심어주는%20계기가%20되었습니다.%20크고%20작은%20비행기%20문제로%20상당히%20지쳐있던%20친구와%20저였지만%20약%202주간의%20여행을%20마무리하는%20마지막%20일정,%20퀸즈타운에서의%205박%206일만큼은%20후회없이%20보내자고%20다짐하며%20도시로%20들어섰습니다.%20때문에%20숙소도%20여지껏%20머물렀던%20유스호스텔이%20아닌,%20'에어비앤비'를%20통해%20예쁜%20독채를%20예약했답니다(❁´▽%60❁)!!
퀸즈타운%20Queenstown%201.26-1.31
숙소뷰가%20정말%20예뻤습니다만,%20시내에서%20다소%20멀고%20엄청난%20오르막길이%20있어%20캐리어를%20끌고%20걸어가기엔%20무리가%20있었습니다.%20(죽을뻔함)
처음엔%20버스를%20타는%20방법을%20몰라,%20숙소까지%20무작정%20걸어갔던%20기억이%20있습니다.%20뜨거운%20햇살을%20맞으며%202주간의%20짐이%20들어있는%20무거운%20캐리어를%20끌고서%20말이죠.%20정말…%20말그대로%20지옥이었습니다.
숙소에%20도착하자마자%20대충%20짐을%20풀고%20숙소%20근처를%20구경할새도%20없이%20곧바로%20잠이%20

1130 1130
1131 1131
1132 1132
1133 1133
1134 1134
1135 1135
1136 1136
1137 1137
1138 1138
1139 1139
1140 1140
1141 1141
1142 1142
1143 1143
1144 1144
1145 1145
1146 1146
1147 1147
1148 1148
1149 1149
1150 1150
1151 1151
1152 1152
1153 1153
1154 1154
1155 1155
1156 1156
1157 1157
1158 1158
1159 1159
1160 1160
1161 1161
1162 1162
1163 1163
1164 1164
1165 1165
1166 1166
1167 1167
1168 1168
1169 1169
1170 1170
1171 1171
1172 1172
1173 1173
1174 1174
1175 1175
1176 1176
1177 1177
1178 1178
1179 1179
1180 1180
1181 1181
1182 1182
1183 1183
1184 1184
1185 1185
1186 1186
1187 1187
1188 1188
1189 1189
1190 1190
1191 1191
1192 1192
1193 1193
1194 1194
1195 1195
1196 1196
1197 1197
1198 1198
1199 1199
1200 1200
1201 1201
1202 1202
1203 1203
1204 1204
1205 1205
1206 1206
1207 1207
1208 1208
1209 1209
1210 1210
1211 1211
1212 1212
1213 1213
1214 1214
1215 1215
1216 1216
1217 1217
1218 1218
1219 1219
1220 1220
1221 1221
1222 1222
1223 1223
1224 1224
1225 1225
1226 1226
1227 1227
1228 1228
1229 1229


KeyboardInterrupt: 

In [17]:
df

,url,본문
0,https://blog.naver.com/jinidag/222104033228,"추석연휴 가볼만한곳 야외 나들이\n""이글은 한국민속촌으로부터 금전적 지원을 받아 작..."
1,https://blog.naver.com/taya2765/222104032204,복천박물관\n부산 동래구 복천로 63\n매일 09:00-18:00 (매주 마지막주 ...
2,https://blog.naver.com/yeonjae1213/222104028784,오늘 아침에 일어나서 엄마랑 극적 화해하고 기분 좋았다. 그런데 기상은 늦었다 10...
3,https://blog.naver.com/sunnybridge_/222104026647,호주 워킹홀리데이 상반기 시즌은 사진이 많이 없어서\n포스팅 할게 많이 없다 ㅠㅠ\...
4,https://blog.naver.com/lby56/222104022581,블로그\n공지 목록\n공지글\n글 제목 작성일\n(12)\n공지 이병렬 저서 출간 ...
5,https://blog.naver.com/qkrtiger/222104017905,컨디션 회복을 위한(?)\n걷기 ...\n아점 후\n형산강변 어싱 ㄱㄱ\n유강방면으...
6,https://blog.naver.com/tnqls11004/222104016240,안녕하세요. 강서구 SNS 서포터즈 '새로미 프렌즈'의 뚜비입니다! 오늘은 강서구에...
7,https://blog.naver.com/019088/222104016283,서로이웃 추가\n이번 달에 새롭게 추가되신 분들은 총 15분입니다. 블로그 운영하는...
8,https://blog.naver.com/jjggrr202/222104015943,blog\n일본 성지순례기(2018) 16개의 글\n일본 성지순례기(2018)\n목...


In [9]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))


214
28
0
